In [1]:
import alpaca_trade_api as tradeapi
import numpy as np
from scipy import optimize as opt
import pandas as pd
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
from bs4 import BeautifulSoup
import requests
from math import log
from sklearn.linear_model import LinearRegression
import datetime 
import workdays as wkdy

/mnt/c/Users/natha/Documents/TourdeFinance/trading_env/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
with open("keys.txt") as keys_file :
    GENERAL_API_KEY = keys_file.readline().strip()
    SECRET_API_KEY = keys_file.readline().strip()
ALPACA_URL_BASE  = "https://paper-api.alpaca.markets"
ALPACA_DATA_URL_BASE = "https://data.alpaca.markets/v1"

In [4]:
api  = tradeapi.REST(GENERAL_API_KEY, SECRET_API_KEY, ALPACA_URL_BASE, api_version = 'v2')
account = api.get_account()

In [5]:
#use get_available_datasets() to see all the different Fama French datasets available
#download the Fama French data
ff5 = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench')
ff5 = ff5[0][::-1] #change it from a dict to a dataframe and then change it 
                   #so it's from newest to oldest
ff5.reset_index(inplace=True)

In [6]:
ff5

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2020-02-28,-0.78,0.06,-0.78,0.32,-0.41,0.006
1,2020-02-27,-4.22,0.69,0.14,-0.35,-0.09,0.006
2,2020-02-26,-0.52,-0.79,-1.26,-0.52,-0.12,0.006
3,2020-02-25,-3.09,-0.34,-0.72,-0.63,0.03,0.006
4,2020-02-24,-3.38,0.15,-0.04,-0.37,0.20,0.006
...,...,...,...,...,...,...,...
1220,2015-04-24,0.17,-0.54,-0.28,0.79,-0.52,0.000
1221,2015-04-23,0.29,0.30,-0.22,-0.05,-0.19,0.000
1222,2015-04-22,0.46,-0.40,0.16,-0.05,-0.02,0.000
1223,2015-04-21,-0.10,0.12,-0.76,0.11,-0.24,0.000


In [7]:
# get list of S&P 500 tickers, code courtesy of Will Everett
page = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
soup = BeautifulSoup(page.content, 'html.parser')
sp500_tickers = [row.find('td').get_text().replace('\n','')
                 for row in soup.find('tbody').find_all('tr')[1:]]
sp500_tickers.remove('BRK.B')
sp500_tickers.remove('BF.B')
sp500_tickers.sort()

In [8]:
#get historical data on the S&P 500 stocks using the Alpaca API
days_of_data = 1000
open_close = pd.DataFrame()
for i in range(0,len(sp500_tickers),100) :
    data = api.get_barset(sp500_tickers[i:i+99],'day',limit=days_of_data)
    for ticker in sp500_tickers[i:i+99] :
        opin = []
        close = []
        for day in range(days_of_data - 1, -1, -1) :
            try :
                if (data[ticker][day].o == 0) :
                    opin.append(np.nan)
                else :
                    opin.append(data[ticker][day].o)
                close.append(data[ticker][day].c)
            except :
                opin.append(np.nan)
                close.append(np.nan)
        open_close[ticker + ' Open'] = opin
        open_close[ticker + ' Close'] = close

In [9]:
open_close

,A Open,A Close,AAL Open,AAL Close,AAP Open,AAP Close,AAPL Open,AAPL Close,ABBV Open,ABBV Close,...,XYL Open,XYL Close,YUM Open,YUM Close,ZBRA Open,ZBRA Close,ZION Open,ZION Close,ZTS Open,ZTS Close
0,78.06,78.750,11.91,11.0601,112.84,117.77,287.38,286.72,82.73,81.800,...,66.76,66.58,77.58,80.06,199.71,201.61,27.87,26.9300,125.22,127.46
1,77.44,77.060,12.52,12.2800,110.95,111.98,282.40,284.45,80.51,81.870,...,68.53,67.12,77.19,77.63,202.34,196.81,28.36,28.0300,125.42,123.94
2,77.30,78.790,12.22,11.9500,111.24,115.46,280.00,287.07,80.62,82.180,...,69.83,70.37,78.30,79.86,204.37,206.88,30.57,29.5500,126.42,127.67
3,77.44,76.210,12.90,11.5600,109.02,110.01,268.31,273.23,80.66,80.380,...,68.87,68.11,79.78,76.99,201.02,197.91,30.81,30.0800,128.01,123.02
4,76.43,78.280,12.64,12.5100,104.62,106.98,268.70,267.96,78.56,79.725,...,69.68,69.34,77.59,80.25,198.83,201.26,30.00,31.0000,126.55,128.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,41.32,41.240,34.20,34.5800,156.34,155.43,94.20,95.18,56.80,57.520,...,42.16,41.77,80.61,79.71,62.71,63.19,27.43,27.5000,47.89,48.13
996,41.20,41.710,34.99,33.8800,155.98,156.09,93.96,93.64,56.62,57.220,...,42.58,42.17,80.18,80.98,62.77,62.56,27.60,27.5100,47.20,48.03
997,40.92,40.930,35.16,34.0300,157.58,155.86,93.99,93.74,61.03,56.810,...,42.84,42.91,80.39,79.56,63.37,62.80,27.45,27.8400,47.16,47.05
998,41.25,41.190,37.04,35.2900,158.07,159.28,97.61,94.82,58.56,61.240,...,42.47,42.88,80.99,80.45,63.97,63.52,26.42,27.4500,47.23,47.21


In [10]:
#remove the tickers from our list that for whatever reason aren't listed in our DataFrame as having data
for tkr in sp500_tickers :
    try :
        open_close[tkr + ' Open'][0]
    except :
        sp500_tickers.remove(tkr)

In [12]:
# Our strategy will be to only consider the 10% of the S&P 500 that was hit hardest by the COVID-19 crash
# First, we calculate the number of days between today and when the stock market first got hit (Feb 19th)
today = datetime.date.today()
days_since_crash = wkdy.networkdays(datetime.date(2020, 2, 19), today)
days_since_crash

42

In [13]:
#calculate how much each stock has lost during the pandemic time
hit_hard = []
for tkr in sp500_tickers :
    dif = (open_close[tkr + ' Open'][days_since_crash] -
           open_close[tkr + ' Close'][0]) / open_close[tkr + ' Open'][days_since_crash]
    hit_hard.append(dif)

In [14]:
#function that finds the n greatest numbers in a list
def n_max(lis, n) :
    maxes = [] 
    for i in range(0, n):  
        max1 = 0
        for j in range(len(lis)):      
            if lis[j] >= max1: 
                try :
                    maxes.index(lis[j])
                except ValueError :
                    max1 = lis[j]
        maxes.append(max1)
    return maxes

In [15]:
#pull out the 50 companies whose stock suffered the greatest lost
discounted_ten_percent = n_max(hit_hard, 50)
discounted_indicies = []
for i in range(len(discounted_ten_percent)) :
    discounted_indicies.append(hit_hard.index(discounted_ten_percent[i]))
len(discounted_indicies)

50

In [16]:
#find their associated ticker and add it to the list "portfolio_stocks"
portfolio_stocks = []
for index in discounted_indicies :
    portfolio_stocks.append(sp500_tickers[index])
portfolio_stocks

['IR',
 'NCLH',
 'APA',
 'CCL',
 'OXY',
 'HAL',
 'RCL',
 'NBL',
 'ADS',
 'DVN',
 'OKE',
 'UAL',
 'MRO',
 'HP',
 'AAL',
 'MPC',
 'FANG',
 'SPG',
 'DAL',
 'BA',
 'KSS',
 'SLB',
 'CMA',
 'DFS',
 'GPS',
 'SYF',
 'FTI',
 'ALK',
 'MGM',
 'JWN',
 'XRX',
 'CFG',
 'KIM',
 'PVH',
 'LNC',
 'TDG',
 'NOV',
 'AIG',
 'EXPE',
 'DRI',
 'EOG',
 'GE',
 'UNM',
 'IVZ',
 'COF',
 'FLS',
 'KEY',
 'LYV',
 'FITB',
 'COTY']

In [17]:
#calculate the days between now and when the Fama French data starts (it's offset by about a month or so)
today = datetime.date.today()
ff5end_date = ff5['Date'][0]
#presidentsday = [datetime.date(2020, 2, 17)]
days_offset = wkdy.networkdays(ff5end_date.date(), today)

In [18]:
#calculate the excess returns for each stock on each day of Fama French data given
#the excess returns are how much more the stock made on that day than the risk free rate ('RF' in the ff5 dataframe)
excess_returns = pd.DataFrame()
num_data = days_of_data - days_offset
excess_returns['Date'] = ff5['Date'][:num_data]
for tkr in portfolio_stocks :
    ex_returns = []
    for day in range(days_offset, days_of_data) :
        daily_ret = (open_close[tkr + ' Close'][day] -
                     open_close[tkr + ' Open'][day] -
                     ff5['RF'][day]) / open_close[tkr + ' Open'][day]
        ex_returns.append(daily_ret)
    excess_returns[tkr] = ex_returns

In [19]:
excess_returns

,Date,IR,NCLH,APA,CCL,OXY,HAL,RCL,NBL,ADS,...,EOG,GE,UNM,IVZ,COF,FLS,KEY,LYV,FITB,COTY
0,2020-02-28,-0.001841,-0.084580,-0.041804,-0.083660,-0.057237,-0.043994,-0.089518,-0.023980,-0.035285,...,-0.066834,-0.036588,-0.023434,-0.030489,-0.015233,-0.016702,-0.028901,-0.041124,-0.025110,-0.042557
1,2020-02-27,-0.032424,-0.085355,-0.063363,-0.057000,-0.084876,-0.047419,-0.084537,-0.056053,-0.053011,...,-0.044989,-0.054804,-0.074358,-0.057957,-0.058019,-0.050841,-0.051314,-0.082374,-0.045362,-0.025746
2,2020-02-26,0.002427,-0.042841,-0.008383,-0.041036,-0.025333,-0.020543,-0.033777,-0.043347,-0.002889,...,-0.019992,0.014029,-0.048188,-0.002143,0.006805,-0.014835,-0.001368,-0.042971,-0.006877,-0.018899
3,2020-02-25,0.003404,-0.024231,0.008671,-0.010116,-0.010247,-0.017837,-0.025230,-0.015680,0.001706,...,-0.008895,-0.016546,0.004827,-0.022974,-0.013326,0.003797,-0.011368,-0.017300,-0.006288,-0.011961
4,2020-02-24,0.005606,-0.058186,-0.000909,-0.002956,0.006066,-0.026021,-0.009795,-0.029125,0.014909,...,-0.010496,-0.010681,0.019425,0.012584,0.022558,0.021914,0.017497,-0.005602,0.013039,-0.002271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,2016-05-05,0.006434,0.015371,-0.026677,0.008950,0.001299,-0.005377,0.007539,-0.014582,-0.004782,...,-0.011171,-0.000685,-0.002942,0.007428,-0.008007,0.001207,-0.001706,0.005507,-0.009945,-0.070806
961,2016-05-04,-0.006402,0.020664,-0.025510,0.003831,0.002078,0.013208,-0.012768,0.012472,-0.008530,...,0.019197,0.007469,-0.000029,-0.004846,0.055509,0.029291,-0.001687,-0.008710,0.005395,0.071369
962,2016-05-03,0.007916,-0.012343,0.020184,-0.002868,-0.018672,0.005523,-0.015678,0.001915,0.025976,...,-0.014762,0.000293,0.010709,-0.013189,-0.051528,-0.013492,-0.000874,0.015694,-0.006026,-0.041561
963,2016-05-02,-0.024918,-0.013780,0.017836,-0.001660,0.010018,-0.001034,0.019867,-0.026701,-0.003971,...,-0.023387,0.005498,0.011523,0.013278,-0.002732,0.016553,0.006325,-0.003738,-0.003825,0.003304


In [30]:
#calculate the betas by linearly regressing the excess returns of each stock with the returns of the different risk factors
#this will be our A matrix
factors = ['Mkt-RF','SMB','HML','RMW','CMA']
betas = pd.DataFrame()
X = pd.DataFrame(ff5[factors][0:num_data], columns=factors)
for tkr in portfolio_stocks :
    y = excess_returns[tkr]
    model = LinearRegression().fit(X, y)
    betas[tkr] = model.coef_

In [21]:
#calculate the premiums for each risk factor
#the premium is how much that trend returns on average (e.g. maybe the market returns an average of 8% each year)
#take a discounted average of the log of the returns, then exponentiate the average
wgts = []
discount_factor = 0.999
for i in range(num_data) :
    wgts.append(discount_factor ** i)
    
log_returns = pd.DataFrame()
for factor in factors :
    log_ret = []
    for i in range(num_data) :
        val = log(ff5[factor][i] + 100) #add 100 so they're all positive, can't log a negative number 
        log_ret.append(val)             
    log_returns[factor + ' Log Returns'] = log_ret

log_premiums = []
for factor in factors :
    lp = np.average(log_returns[factor + ' Log Returns'], weights = wgts)
    log_premiums.append(lp)

premiums = np.exp(log_premiums)
premiums -= 100

In [31]:
#calculate the expects returns for each stock using the risk free rate, betas, and premiums
#these with be the coefficients of our objective function
expected_returns = []
for tkr in portfolio_stocks :
    exp_re = ff5['RF'][0] #expected returns is the risk-free rate, plus the betas times the premiums
    for factor in range(len(factors)) :
        exp_re += betas[tkr][factor]*premiums[factor]
    expected_returns.append(exp_re)

In [34]:
#multiply each of the expected returns and each of the betas by the current stock price
#also add the constraint that x times the different stock prices must be less than total capital onto the bottom of the betas df
stock_prices = pd.DataFrame()
for i in range(len(portfolio_stocks)) :
    tkr = portfolio_stocks[i]
    betas[tkr] = betas[tkr] * open_close[tkr + ' Close'][0]
    expected_returns[i] *= open_close[tkr + ' Close'][0]
    expected_returns[i] *= -1 #we have to reverse our obj funct because it minimizes
    stock_prices[tkr] = [open_close[tkr + ' Close'][0]]
betas = betas.append(stock_prices)

In [41]:
#last, create our b vector
#this represents the amount of risk we're willing to take in each direction, along with our total capital
b = [3,5,7,8,9,1000]

In [42]:
#use the linear program solver
x = opt.linprog(c = expected_returns, A_ub = betas, b_ub = b)
x

     con: array([], dtype=float64)
     fun: -6.104114735212041
 message: 'Optimization terminated successfully.'
     nit: 10
   slack: array([2.06605652e+00, 5.65325597e+00, 5.40179523e+00, 7.64779170e+00,
       1.61816611e+01, 1.00795035e-06])
  status: 0
 success: True
       x: array([2.47183715e-06, 6.08408281e-06, 1.11835932e-05, 4.63407730e-06,
       7.15148329e-06, 1.02249903e-05, 2.25667292e-06, 1.08815977e-05,
       1.67922710e-06, 6.58971525e-06, 2.51349633e-06, 1.75959346e-06,
       3.23693853e-05, 3.91688252e-06, 4.89791465e-06, 5.01398351e-06,
       2.18147617e-06, 9.65552515e-07, 2.23188151e-06, 3.92708408e-07,
       3.24784246e-06, 3.99679541e-06, 2.91658749e-06, 2.20057138e-06,
       3.39571561e-05, 4.68434910e-06, 9.85247132e-06, 1.72713238e-06,
       3.40079017e-06, 3.37813099e-06, 3.29113811e-06, 3.81606257e-06,
       6.71378736e-06, 2.39062915e-06, 4.00615378e-06, 2.31068482e-07,
       4.38134386e-04, 2.31497735e-06, 2.15264308e-06, 8.80127001e-07,
     

In [44]:
np.sum(x.x)

175.13055451451172

In [46]:
portfolio_stocks[49]
#and.... it tells us to put all our money into COTY a NY cosmetics company

'COTY'

In [ ]:
# I think it's cx - k + c2x2 - k2 etc

In [6]:
api.submit_order(symbol = 'AAPL',
                 qty = 3,
                 side = 'buy',
                 type = 'market',
                 time_in_force = 'day')

Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': 'a192e8d2-b3b1-4a79-8054-dfd649e61a56',
    'created_at': '2020-04-03T21:32:04.784938Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '77439e17-4f58-401c-a38c-89029d04868c',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '3',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-04-03T21:32:04.777419Z',
    'symbol': 'AAPL',
    'time_in_force': 'day',
    'type': 'market',
    'updated_at': '2020-04-03T21:32:04.784938Z'})

In [ ]:
clock = api.get_clock()
if clock.is_open :
    #trade away! 

In [7]:
ff5

{0:             Mkt-RF   SMB   HML   RMW   CMA     RF
 Date                                             
 2015-03-09    0.37  0.09 -0.02  0.26 -0.03  0.000
 2015-03-10   -1.63  0.43 -0.47  0.09 -0.14  0.000
 2015-03-11   -0.04  0.62  0.52 -0.47 -0.03  0.000
 2015-03-12    1.28  0.40  0.50 -0.06  0.23  0.000
 2015-03-13   -0.57  0.19 -0.04 -0.10 -0.13  0.000
 ...            ...   ...   ...   ...   ...    ...
 2020-01-27   -1.56  0.27 -0.48 -0.59  0.05  0.006
 2020-01-28    1.02 -0.15 -0.42  0.17 -0.10  0.006
 2020-01-29   -0.10 -0.44 -0.95  0.25  0.19  0.006
 2020-01-30    0.34 -0.64  0.67 -0.39 -0.11  0.006
 2020-01-31   -1.74 -0.52 -0.34 -0.54 -0.69  0.006
 
 [1235 rows x 6 columns],
 'DESCR': 'F-F Research Data 5 Factors 2x3 daily\n-------------------------------------\n\nThis file was created by CMPT_ME_BEME_OP_INV_RETS_DAILY using the 202001 CRSP database. The 1-month TBill return is from Ibbotson and Associates, Inc.\n\n  0 : (1235 rows x 6 cols)'}